<a href="https://colab.research.google.com/github/rafaqatali69/Batch-57/blob/main/Rag_project_Ziakhan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Google key

In [ ]:
from google.colab import userdata
GEM_API_KEY =userdata.get('GOOGLE_API_KEY')
print(GEM_API_KEY)

**installion of langchain chroma and langchain community**

In [ ]:
%pip install -qU pypdf
!pip install langchain_community pypdf -q langchain_google_genai  langchain_chroma
!pip install  --upgrade -q google-auth google-auth-oauthlib google-auth-httplib2  google-cloud

**Upload the Document file in PDF format**

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/K2. Life and death on the worlds most dangerous mountain - Ed Viesturs.pdf")
data = loader.load()


**Print the content of File**

In [ ]:
print(data[0].metadata)
print(data[0].page_content)

**split the document by Rag and langchain calling**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("total number of documents : " ,len(docs))

**Document Lengh**

In [ ]:
len(data)

**Download test.jsn file and start Embedding of Documents**

https://drive.google.com/file/d/1afKObnhCZ9UeXyMuqiV2t4BJ9Nkvgm4D/view

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/test.json"

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

In [ ]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=GEM_API_KEY)

In [ ]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})


**LLM Model for Langchain calling**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" , temperature = 0.3 , max_tokens=1000)


**Retriwal of Data**

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


**Prompt and Responses by Question/Answers**

In [ ]:
system_prompt = (
    "As You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input" : "Did you know rafaqat ali"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input" : "Tell me some detail of K2 Mountain"})
print(response["answer"])